In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

edge_driver_path = 'D:\Vlad\Все ДЗ(2)\Python\expl\msedgedriver.exe'

edge_options = Options()
edge_options.add_argument('--headless')
edge_options.add_argument('--disable-gpu')
edge_options.add_argument('--no-sandbox')

service = Service(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

def collect_news_data(keyword, months=6):
    base_url = 'https://www.bbc.com'
    search_url = f'{base_url}/search?q={keyword}'

    driver.get(search_url)
    time.sleep(2)

    news_data = []

    while True:
        articles = driver.find_elements(By.CSS_SELECTOR, 'article')
        for article in articles:
            try:
                title_element = article.find_element(By.CSS_SELECTOR, 'h1, h2, h3')
                title = title_element.text
                link = title_element.find_element(By.TAG_NAME, 'a').get_attribute('href')
                date_element = article.find_element(By.CSS_SELECTOR, 'time')
                date = date_element.get_attribute('datetime')
                news_data.append({'title': title, 'link': link, 'date': date})
            except Exception as e:
                continue

        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.next')
            next_button.click()
            time.sleep(2)
        except Exception as e:
            break

    return pd.DataFrame(news_data)

keyword = 'plane'
news_df = collect_news_data(keyword)

news_df['date'] = pd.to_datetime(news_df['date'])

end_date = pd.Timestamp.today()
start_date = end_date - pd.DateOffset(months=6)
filtered_df = news_df[(news_df['date'] >= start_date) & (news_df['date'] <= end_date)]

filtered_df['month'] = filtered_df['date'].dt.to_period('M')
monthly_counts = filtered_df.groupby('month').size()

plt.figure(figsize=(10, 5))
monthly_counts.plot(kind='bar', color='skyblue')
plt.title(f'Частота появи фрази "{keyword}" у новинах BBC за останні 6 місяців')
plt.xlabel('Місяць')
plt.ylabel('Кількість новин')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()